In [1]:
from google.cloud import vision
import io

# Initialize client using the service account JSON key directly
client = vision.ImageAnnotatorClient.from_service_account_file('firebaseSecretKey.json')


In [2]:
with io.open("screenshots/dr-arunkumar.png", 'rb') as image_file:
    content = image_file.read()

image = vision.Image(content=content)

# Perform text detection
response = client.text_detection(image=image)
texts = response.text_annotations

In [11]:
import os

if texts:
    extracted_text = texts[0].description
    os.makedirs("extracted_texts", exist_ok=True)
    with open("extracted_texts/dr-arunkumar.txt", "w", encoding="utf-8") as f:
        f.write(extracted_text)
    print("✅ Text extracted and saved to extracted_texts/dr-arunkumar.txt")
else:
    print("No text found.")

# Handle errors
if response.error.message:
    raise Exception(f'{response.error.message}')

✅ Text extracted and saved to extracted_texts/dr-arunkumar.txt


In [12]:
from transformers import AutoTokenizer
import json
import os

os.makedirs("chunks", exist_ok=True)

# Load Hugging Face tokenizer (using a common one)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

# Read the extracted text file
with open("extracted_texts/dr-arunkumar.txt", "r", encoding="utf-8") as f:
    text = f.read()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Rohan\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Rohan\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [13]:
def chunk_text(text, max_tokens=300):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i+max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)
    return chunks

# Chunk the text
chunks = chunk_text(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (2363 > 512). Running this sequence through the model will result in indexing errors


In [16]:
import json

chunks_data = {
    "source": "dr-arunkumar.txt",
    "chunks": [{"id": i+1, "text": chunk} for i, chunk in enumerate(chunks)]
}
os.makedirs("chunks", exist_ok=True)
# Save chunks to a JSON file
with open("chunks/dr-arunkumar_chunks.json", "w", encoding="utf-8") as f:
    json.dump(chunks_data, f, indent=4, ensure_ascii=False)

print("🎉 All chunks saved to chunks/dr-arunkumar_chunks.json")

NameError: name 'json' is not defined